In [1]:
#Dashboard devolped by Moezzibadi - DS2E
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
import sys
sys.modules[__name__].__dict__.clear()
### Run ngrok to tunnel Dash app port 8050 to the outside world. 
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8050 &')

### Get the public URL where you can access the Dash app. Copy this URL.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Install Dash
#!pip install dash-html-components==0.13.2  # HTML components
#!pip install dash-core-components==0.39.0  # Supercharged components
#!pip install dash-table==3.1.7  # Interactive DataTable component (new!)
#! pip install pyreadstat
##!pip3 uninstall dash
#!pip3 install dash==2.0.0

import pandas as pd
import pyreadstat
import numpy as np
import plotly.graph_objects as go

dtafile = './Data/labor_flows_ind2_Occ_CZ.dta'
df = pd.read_stata(dtafile)
df['prev_ctg']=df['prev_unit'].str.slice(2,3)
df['prev_reg']=df['prev_unit'].str.slice(start=3)
df['prev_unit']=df['prev_unit'].str.slice(stop=2)  
df['new_ctg']=df['new_unit'].str.slice(2,3)
df['new_reg']=df['new_unit'].str.slice(start=3)
df['new_unit']=df['new_unit'].str.slice(stop=2)  
df.to_csv('file1.csv')
df.new_reg=df.new_reg.astype(int)
df.prev_reg=df.prev_reg.str.zfill(8).astype(int)

df22 = pd.read_csv('./Data/communes-departement-region.csv')
df22 = df22[['nom_commune','nom_region']].dropna()
df22 = df22.rename(columns={'nom_commune': 'Commune'})

df2 = pd.read_csv('./Data/Cord4.csv', delimiter=';')
df2 = df2.rename(columns={'LIBGEO':'Commune'})
df2['Commune'] = df2['Commune'].str.replace(', France', '')
df2 = df2.rename(columns={'ZE2020': 'new_reg'})
mergedStuff11 = df2.merge(df22,how='left', left_on='Commune', right_on='Commune')
pos = [5,7,8,9, 12, 47,355, 113, 410 , 411, 412,413,414,415,416,417,418,419,420,421,422]
mergedStuff11 = mergedStuff11.drop([5,7,8,9, 12, 47,355, 113,406,407 ,410 , 411, 412,413,414,415,416,417,418,419,420,421,422]) 
mergedStuff11 = mergedStuff11.drop_duplicates()

mergedStuff = df.merge(mergedStuff11,how='left', left_on='new_reg', right_on='new_reg')
mergedStuff = mergedStuff.drop(['LIBZE2020','Unnamed: 0', 'index','CODGEO'], axis=1)
mergedStuff = mergedStuff.rename(columns={'latitude': 'new_lat'})
mergedStuff = mergedStuff.rename(columns={'longitude': 'new_long'})
mergedStuff = mergedStuff.rename(columns={'nom_region': 'new_region'})
mergedStuff = mergedStuff.rename(columns={'Commune': 'new_Commune'})
mergedStuff = mergedStuff[pd.notna(mergedStuff['new_unit'])]
mergedStuff = mergedStuff[pd.notna(mergedStuff['new_Commune'])]

mergedStuff = mergedStuff.merge(mergedStuff11,how='left', left_on='prev_reg', right_on='new_reg')
mergedStuff = mergedStuff.drop(['LIBZE2020','Unnamed: 0', 'index','CODGEO','new_reg_y'], axis=1)
mergedStuff = mergedStuff.rename(columns={'latitude': 'pre_lat','Commune': 'prev_Commune'})
mergedStuff = mergedStuff.rename(columns={'nom_region': 'prev_region'})
mergedStuff = mergedStuff.rename(columns={'longitude': 'pre_long','new_reg_x' : 'new_reg_'})
mergedStuff

data = mergedStuff[(mergedStuff.switch== 1.0) & (mergedStuff.prev_unit != mergedStuff.new_unit) ]
NaN = np.nan
data["new_ctg2"] = NaN
data['new_ctg2'] = pd.factorize(data.new_ctg)[0]

data["pre_ctg2"] = NaN
data['pre_ctg2'] = pd.factorize(data.prev_ctg)[0]

# Dashboard
!pip install jupyter-dash
from jupyter_dash import JupyterDash
app = JupyterDash()
app.css.config.serve_locally = True

from importlib import reload
import logging
reload(logging)
logging.getLogger('werkzeug')
! log.setLevel(logging.ERROR)
import plotly.express as px  # (version 4.7.0 or higher)
#!pip3 uninstall dash
#!pip3 install dash
from dash import dcc  ,Dash, html # pip install dash (version 2.0.0 or higher)
from dash.dependencies import Output, Input

#import dash_core_components as dcc
#import dash_html_components as html
from PIL import Image
import plotly.io as pio
import base64
# Build App

# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([
    html.Div(children=[
 # second column of first row
     html.Div(children=[
        html.Div(children=[ 
          html.H1("Labor flow in france", style={'text-align': 'center'}),
                   dcc.Dropdown(id="slct_year",
                    options=[
                     {"label": "Inflow frequency - mean tenure", "value": 1},
                     {"label": "Outflow frequency - mean tenure", "value": 2},
                     {"label": "Most Labor category of inflow", "value": 3},
                     {"label": "Most Labor category of outflow", "value": 4},
                     {"label": "Île-de-France", "value": 5},
                     {"label": "Occitanie", "value": 6},
                     {"label": "Pays de la Loire", "value": 7},
                     {"label": "Provence-Alpes-Côte d\'Azur", "value": 8},
                     {"label": "Bretagne", "value": 9},
                     {"label": "Bourgogne-Franche-Comté", "value": 10},
                     {"label": "Corse", "value": 11},
                     {"label": "Nouvelle-Aquitaine", "value": 12},
                     {"label": "Normandie", "value": 13},
                     {"label": "Hauts-de-France", "value": 14},
                     {"label": "Centre-Val de Loire", "value": 15}],          
                 multi=False,
                 value=1,
                 style={'width': "60%"}
                 ),
       html.Div(dcc.Graph(id='my_bee_map', figure={}, style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'})),
       html.Div(id='output_container', children=[]),
 ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
         html.Div(children=[  html.H1("Network analysis - centrality", style={'text-align': 'right'}),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Iframe(srcDoc = open('./Data/html_file.html', 'r').read() ,
             style = {'display': 'inline-block',
                     'box-sizing': 'border-box',
                      'horizontal-align': 'center',
                      'width': '180%',
                      'height': 585})
           ], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw'}),
           ]),
 ], className='row'),
])
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')])
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The graph chosen by user : {}".format(option_slctd)
    if option_slctd == 1 : 
        dfff = data
        dfff.pre_Commune = dfff.prev_Commune.str.strip()
        dfff.Commune = dfff.new_Commune.str.strip()
        #df_tenure = dfff.groupby(['pre_Commune','pre_lat'	,'pre_long','Commune','new_lat',	'new_long'])['tenure'].agg(['mean','count']).reset_index()
        df_tenure = dfff.groupby(['new_Commune','new_lat','new_long','new_region'])['tenure'].agg(['mean','count']).reset_index()
        df_tenure['mean_size'] = df_tenure['mean'].fillna(0)*10
        NaN = np.nan
        df_tenure["color"] = NaN
        df_tenure["shape"] = NaN
        df_tenure['count1'] = NaN
        df_tenure['color'][df_tenure['count']<=20]='green'
        df_tenure['color'][df_tenure['count']>20]='blue'
        df_tenure['color'][df_tenure['count']>40]='red'
        df_tenure['shape'][df_tenure['count']<=20]='square'
        df_tenure['shape'][df_tenure['count']>20]='circle'
        df_tenure['shape'][df_tenure['count']>40]='diamond'
        df_tenure['count1'][df_tenure['count']<=20]='Low count'
        df_tenure['count1'][df_tenure['count']>20]='Medium count'
        df_tenure['count1'][df_tenure['count']>40]='High count'
       # Plotly Express
        fig = go.Figure(data=[go.Scattergeo(
            lon=df_tenure['new_long'],
            lat=df_tenure['new_lat'],
            text=df_tenure['new_region'],
            customdata=np.stack((df_tenure['count1'], df_tenure['new_Commune']), axis=-1),
            marker=dict(size=df_tenure['mean_size'],
                        color=df_tenure['color'],
                        symbol=df_tenure['shape'],
                        line=dict(width=3, color='rgba(68, 68, 68, 0)')
                        ))],
         layout=go.Layout(title = 'Inflow count by a destination town',
        height=900,
            geo_scope = 'europe')).update_geos(fitbounds='locations').update_traces(
    hovertemplate="<br>".join([
        "Region: %{text}",
        "Town: %{customdata[1]}",
        "Num.: %{customdata[0]}<br><extra></extra>"
    ])
)
        # Add locations
    elif option_slctd == 2:
        dfff = data
        dfff.pre_Commune = dfff.pre_Commune.str.strip()
        dfff.Commune = dfff.Commune.str.strip()
        df_tenure = dfff.groupby(['prev_Commune','pre_lat','pre_long','prev_region'])['tenure'].agg(['mean','count']).reset_index()
        df_tenure['mean_size'] = df_tenure['mean'].fillna(0)*10
        NaN = np.nan
        df_tenure["color"] = NaN
        df_tenure["shape"] = NaN
        df_tenure['count1'] = NaN
        df_tenure['color'][df_tenure['count']<=20]='green'
        df_tenure['color'][df_tenure['count']>20]='blue'
        df_tenure['color'][df_tenure['count']>40]='red'
        df_tenure['shape'][df_tenure['count']<=20]='square'
        df_tenure['shape'][df_tenure['count']>20]='circle'
        df_tenure['shape'][df_tenure['count']>40]='diamond'
        df_tenure['count1'][df_tenure['count']<=20]='Low count'
        df_tenure['count1'][df_tenure['count']>20]='Medium count'
        df_tenure['count1'][df_tenure['count']>40]='High count'

        # Add locations
        fig = go.Figure(data=[go.Scattergeo(
            lon=df_tenure['pre_long'],
            lat=df_tenure['pre_lat'],
            text=df_tenure['prev_region'],
            customdata=np.stack((df_tenure['count1'], df_tenure['prev_Commune']), axis=-1),
            marker=dict(size=df_tenure['mean_size'],
                        color=df_tenure['color'],
                        symbol=df_tenure['shape'],
                        line=dict(width=3, color='rgba(68, 68, 68, 0)')
                        ))],
         layout=go.Layout(title = 'Inflow count by a destination town',
        height=900,
            geo_scope = 'europe')).update_geos(fitbounds='locations').update_traces(
    hovertemplate="<br>".join([
        "Region: %{text}",
        "Town: %{customdata[1]}",
        "Num.: %{customdata[0]}<br><extra></extra>"
    ])
)
    elif option_slctd == 3:
        dfff = data
        dfff.pre_Commune = dfff.pre_Commune.str.strip()
        dfff.Commune = dfff.Commune.str.strip()
        df_tenure = dfff.groupby(['new_Commune','new_lat','new_long','new_region'])['new_ctg2'].agg(['mean','count']).reset_index()
        NaN = np.nan
        df_tenure['mean_size'] = df_tenure['mean'].fillna(0)*10
        df_tenure["color"] = NaN
        df_tenure["shape"] = NaN
        df_tenure["Qual"] = NaN
        df_tenure['color'][:]='red'
        df_tenure['color'][df_tenure['mean']>0.5]='green'
        df_tenure['color'][df_tenure['mean']>1.2]='blue'
        df_tenure['shape'][:]='square'
        df_tenure['shape'][df_tenure['mean']>0.5]='circle'
        df_tenure['shape'][df_tenure['mean']>1.2]='diamond'
        df_tenure['Qual'][:]='Low'
        df_tenure['Qual'][df_tenure['mean']>0.5]='Med.'
        df_tenure['Qual'][df_tenure['mean']>1.2]='High'
        # Add locations
        fig = go.Figure(data=[go.Scattergeo(
            lon=df_tenure['new_long'],
            lat=df_tenure['new_lat'],
            text=df_tenure['new_region'],
            customdata=np.stack((df_tenure['Qual'], df_tenure['new_Commune']), axis=-1),
            marker=dict(size=df_tenure['mean_size'],
                        color=df_tenure['color'],
                        symbol=df_tenure['shape'],
                        line=dict(width=3, color='rgba(68, 68, 68, 0)')
                        ))],
         layout=go.Layout(title = 'Average labor category by an inflow town',
        height=900,
            geo_scope = 'europe')).update_geos(fitbounds='locations').update_traces(
    hovertemplate="<br>".join([
        "Region: %{text}",
        "Town: %{customdata[1]}",
        "Lab. category: %{customdata[0]}<br><extra></extra>"
    ])
)            
    elif option_slctd == 4:
        dfff = data
        dfff.pre_Commune = dfff.pre_Commune.str.strip()
        dfff.Commune = dfff.Commune.str.strip()
        df_tenure = dfff.groupby(['prev_Commune','pre_lat','pre_long','prev_region'])['new_ctg2'].agg(['mean','count']).reset_index()
        NaN = np.nan
        df_tenure['mean_size'] = df_tenure['mean'].fillna(0)*10
        df_tenure["color"] = NaN
        df_tenure["shape"] = NaN
        df_tenure["Qual"] = NaN
        df_tenure['color'][:]='red'
        df_tenure['color'][df_tenure['mean']>0.5]='green'
        df_tenure['color'][df_tenure['mean']>1.2]='blue'
        df_tenure['shape'][:]='square'
        df_tenure['shape'][df_tenure['mean']>0.5]='circle'
        df_tenure['shape'][df_tenure['mean']>1.2]='diamond'
        df_tenure['Qual'][:]='Low'
        df_tenure['Qual'][df_tenure['mean']>0.5]='Med.'
        df_tenure['Qual'][df_tenure['mean']>1.2]='High'
        # Add locations
        fig = go.Figure(data=[go.Scattergeo(
            lon=df_tenure['pre_long'],
            lat=df_tenure['pre_lat'],
            text=df_tenure['prev_region'],
            customdata=np.stack((df_tenure['Qual'], df_tenure['prev_Commune']), axis=-1),
            marker=dict(size=df_tenure['mean_size'],
                        color=df_tenure['color'],
                        symbol=df_tenure['shape'],
                        line=dict(width=3, color='rgba(68, 68, 68, 0)')
                        ))],
         layout=go.Layout(title = 'Average labor category by an inflow town',
        height=900,
            geo_scope = 'europe')).update_geos(fitbounds='locations').update_traces(
    hovertemplate="<br>".join([
        "Region: %{text}",
        "Town: %{customdata[1]}",
        "Lab. category: %{customdata[0]}<br><extra></extra>"
    ])
)
    elif option_slctd > 4:
        dfff = data
        dfff.pre_Commune = dfff.pre_Commune.str.strip()
        dfff.Commune = dfff.Commune.str.strip()
        df_tenure = dfff.groupby(['prev_Commune','pre_lat','pre_long','prev_region', 'new_long','new_lat','new_Commune'])['new_ctg2'].agg(['mean','count']).reset_index()
        NaN = np.nan
        NaN = np.nan
        df_tenure['mean_size'] = df_tenure['mean'].fillna(0)*10
        df_tenure["color"] = NaN
        df_tenure["shape"] = NaN
        df_tenure['count1'] = NaN
        df_tenure['color'][df_tenure['count']<=20]='green'
        df_tenure['color'][df_tenure['count']>20]='blue'
        df_tenure['color'][df_tenure['count']>40]='red'
        df_tenure['shape'][df_tenure['count']<=20]='square'
        df_tenure['shape'][df_tenure['count']>20]='circle'
        df_tenure['shape'][df_tenure['count']>40]='diamond'
        df_tenure['count1'][df_tenure['count']<=20]='Low count'
        df_tenure['count1'][df_tenure['count']>20]='Medium count'
        df_tenure['count1'][df_tenure['count']>40]='High count'
        
        xx = NaN
        
        if option_slctd == 5 :
          xx = df_tenure[df_tenure.prev_region=='Pays de la Loire']
          jjj = 'Pays de la Loire'
        if option_slctd == 6 :
          xx = df_tenure[df_tenure.prev_region=='Occitanie']
          jjj = 'Occitanie'
        if option_slctd == 7 :
          xx = df_tenure[df_tenure.prev_region=='Île-de-France']   
          jjj = 'Île-de-France'
        if option_slctd == 8 :
          xx = df_tenure[df_tenure.prev_region=='Provence-Alpes-Côte d\'Azur'] 
          jjj = 'Provence-Alpes-Côte d\'Azur'
        if option_slctd == 9 :
          xx = df_tenure[df_tenure.prev_region=='Bretagne'] 
          jjj = 'Bretagne'
        if option_slctd == 10 :
          xx = df_tenure[df_tenure.prev_region=='Bourgogne-Franche-Comté'] 
          jjj = 'Bourgogne-Franche-Comté'
        if option_slctd == 11 :
          xx = df_tenure[df_tenure.prev_region=='Corse'] 
          jjj = 'Corse'
        if option_slctd == 12 :
          xx = df_tenure[df_tenure.prev_region=='Nouvelle-Aquitaine']  
          jjj =  'Nouvelle-Aquitaine'
        if option_slctd == 13 :
          xx = df_tenure[df_tenure.prev_region=='Normandie'] 
          jjj = 'Normandie'
        if option_slctd == 14 :
          xx = df_tenure[df_tenure.prev_region=='Hauts-de-France'] 
          jjj = 'Hauts-de-France'
        if option_slctd == 15 :
          xx = df_tenure[df_tenure.prev_region=='Centre-Val de Loire'] 
          jjj = 'Centre-Val de Loire'                    
        xx["Nan"]  = NaN
        xx["Zero"]  = 0
        col1 = xx.pre_long
        col2 = xx.new_long
        col22 = xx["prev_Commune"]
        col33 = xx["new_Commune"]
        col3 = xx["Nan"]
        Long1= list(zip(*[col1,col2,col3]))
        Long2 = [i for sub in Long1 for i in sub]
        CCount1= list(zip(*[col22,col33,col33]))
        CCount2 = [i for sub in CCount1 for i in sub]
        col4 = xx.pre_lat
        col5 = xx.new_lat
        Lat1 = list(zip(*[col4,col5,col3]))
        Lat2 = [i for sub in Lat1 for i in sub]
        # Add locations
        fig = go.Figure(data=[go.Scattergeo(
              lon= Long2,
              lat= Lat2,
              mode='lines',
              line=dict(width=2, color='blue'),
              opacity=0.8,
              text=CCount2,
              showlegend=False)],
         layout=go.Layout(title = 'Network of origin towns in {}'.format(jjj),
        height=900,
            geo_scope = 'europe')).update_geos(fitbounds='locations')
    return container, fig
# Run app and display result inline in the notebook
app.run_server(mode='external')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/moezzibadi/opt/anaconda3/lib/python3.8/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/Users/moezzibadi/opt/anaconda3/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/Users/moezzibadi/opt/anaconda3/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/moezzibadi/opt/anaconda3/lib/python3.8/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


zsh:1: no such file or directory: ./ngrok


FileNotFoundError: [Errno 2] No such file or directory: './Data/labor_flows_ind2_Occ_CZ.dta'

In [ ]:
#! kill -9 $(ps -A | grep python | awk '{print $1}')